In [2]:
context = {}

context['symbols'] = [
    'BP',
    'CD',
    'CL',
    'ED',
    'GC',
    'HG',
    'HO',
    'HU',
    'JY',
    'SB',
    'SF',
    'SP',
    'SV',
    'TB',
    'TY',
    'US'
]

assert(len(context['symbols']) == 16)

In [3]:
from quantopian.research.experimental import continuous_future

context['markets'] = map(
    lambda market: continuous_future(market),
    context['symbols']
)

assert(len(context['markets']) == 16)

In [4]:
from quantopian.research.experimental import history

fields = ['high', 'low', 'close_price']
start_date = '2017-02-27'
end_date = '2017-03-31'
frequency = 'daily'

context['prices'] = history(
    context['markets'],
    fields=fields,
    frequency=frequency,
    start_date=start_date,
    end_date=end_date
)

assert(context['prices'].shape == (3, 25, 16))

In [5]:
context['prices'].dropna(axis=2, inplace=True)

assert(context['prices'].shape == (3, 25, 14))

In [6]:
if len(context['prices'].items) == len(fields):
    context['prices'] = context['prices'].transpose(2, 1, 0)
    context['prices'] = context['prices'].reindex()

assert(context['prices'].shape == (14, 25, 3))

In [7]:
from talib import ATR

context['average_true_range'] = {}

rolling_window = 21
moving_average = 20

for market in context['prices'].items:
    context['average_true_range'][market] = ATR(
        context['prices'][market].high[-rolling_window:],
        context['prices'][market].low[-rolling_window:],
        context['prices'][market].close_price[-rolling_window:],
        timeperiod=moving_average
    )[-1]

context['average_true_range']

{ContinuousFuture(90722903447896064, root_symbol='BP', offset=0, roll_style='volume', adjustment='mul'): 0.010249999999999959,
 ContinuousFuture(90991184285073408, root_symbol='CD', offset=0, roll_style='volume', adjustment='mul'): 0.0043000000000000035,
 ContinuousFuture(90999980378095616, root_symbol='CL', offset=0, roll_style='volume', adjustment='mul'): 1.2441000000000009,
 ContinuousFuture(91554134238494720, root_symbol='ED', offset=0, roll_style='volume', adjustment='mul'): 0.02049999999999912,
 ContinuousFuture(92115984680288256, root_symbol='GC', offset=0, roll_style='volume', adjustment='mul'): 12.243899999999963,
 ContinuousFuture(92401857703510016, root_symbol='HG', offset=0, roll_style='volume', adjustment='mul'): 0.04830000000000001,
 ContinuousFuture(92410653796532224, root_symbol='HO', offset=0, roll_style='volume', adjustment='mul'): 0.033899999999999951,
 ContinuousFuture(92984598866231296, root_symbol='JY', offset=0, roll_style='volume', adjustment='mul'): 0.0,
 Conti

In [8]:
fields = ['contract']
start_date = '2017-03-31'
end_date = '2017-03-31'

context['contracts'] = history(
    context['markets'],
    fields=fields,
    frequency=frequency,
    start_date=start_date,
    end_date=end_date
)

assert(context['contracts'].shape == (1, 1, 16))

In [9]:
if len(context['contracts'].items) == len(fields):
    context['contracts'] = context['contracts'].transpose(2, 1, 0)
    context['contracts'] = context['contracts'].reindex()

assert(context['contracts'].shape == (16, 1, 1))

In [10]:
for market in context['contracts']:
    contract = context['contracts'][market].contract[0]
    
    if contract is not None:
        print(
            contract.symbol,
            contract.tick_size,
            contract.multiplier,
            contract.auto_close_date
        )

(u'BPM17', 0.0001, 62500.0, Timestamp('2017-06-15 00:00:00+0000', tz='UTC'))
(u'CDM17', 0.0001, 100000.0, Timestamp('2017-06-16 00:00:00+0000', tz='UTC'))
(u'CLK17', 0.01, 1000.0, Timestamp('2017-04-18 00:00:00+0000', tz='UTC'))
(u'EDJ17', 0.005, 2500.0, Timestamp('2017-04-12 00:00:00+0000', tz='UTC'))
(u'GCM17', 0.1, 100.0, Timestamp('2017-05-29 00:00:00+0000', tz='UTC'))
(u'HGK17', 0.0005, 25000.0, Timestamp('2017-04-26 00:00:00+0000', tz='UTC'))
(u'HOK17', 0.0001, 42000.0, Timestamp('2017-04-26 00:00:00+0000', tz='UTC'))
(u'JYM17', 5e-07, 12500000.0, Timestamp('2017-06-15 00:00:00+0000', tz='UTC'))
(u'SBK17', 0.0001, 112000.0, Timestamp('2017-04-26 00:00:00+0000', tz='UTC'))
(u'SFM17', 0.0001, 125000.0, Timestamp('2017-06-15 00:00:00+0000', tz='UTC'))
(u'SPM17', 0.1, 250.0, Timestamp('2017-06-13 00:00:00+0000', tz='UTC'))
(u'SVK17', 0.005, 5000.0, Timestamp('2017-04-26 00:00:00+0000', tz='UTC'))
(u'TYM17', 0.015625, 1000.0, Timestamp('2017-05-29 00:00:00+0000', tz='UTC'))
(u'USM17',

In [11]:
for market in context['contracts']:
    contract = context['contracts'][market].contract[0]
    
    if contract is not None:
        dollar_volatility = contract.tick_size * context['average_true_range'][market]
        
        print(market.root_symbol, dollar_volatility)

('BP', 1.0249999999999959e-06)
('CD', 4.3000000000000038e-07)
('CL', 0.012441000000000009)
('ED', 0.00010249999999999561)
('GC', 1.2243899999999963)
('HG', 2.4150000000000004e-05)
('HO', 3.3899999999999951e-06)
('JY', 0.0)
('SB', 5.1999999999999989e-07)
('SF', 6.5500000000000336e-07)
('SP', 1.5539249999999925)
('SV', 0.0013150000000000048)
('TY', 0.0076156249999999818)
('US', 0.037154687500000082)


In [12]:
context['capital_risk_per_trade'] = 0.01
context['starting_cash'] = 1000000
# context['portfolio_value'] = 1100000
context['portfolio_value'] = 900000
context['capital_multiplier'] = 2

profit = context['portfolio_value'] - context['starting_cash']
context['cash'] = context['starting_cash']

if profit < 0:
    context['cash'] = context['starting_cash'] + profit * context['capital_multiplier']

capital_per_trade = context['cash'] * context['capital_risk_per_trade']
trade_size = capital_per_trade / dollar_volatility

int(trade_size)

215316

In [28]:
context['twenty_day_breakout'] = 2
context['fifty_five_day_breakout'] = 55

In [36]:
for market in context['prices'].items:
    print(market.root_symbol, context['prices'][market].high[-context['twenty_day_breakout']:].max())
    print(market.root_symbol, context['prices'][market].low[-context['twenty_day_breakout']:].min())
#     print(market.root_symbol, context['prices'][market].high[-context['twenty_day_breakout']-1:-1])
#     print(market.root_symbol, context['prices'][market].low[-context['twenty_day_breakout']-1:-1])

('BP', 1.2589999999999999)
('BP', 1.2430000000000001)
('CD', 0.754)
('CD', 0.749)
('CL', 50.850000000000001)
('CL', 49.270000000000003)
('ED', 98.724999999999994)
('ED', 98.704999999999998)
('GC', 1256.4000000000001)
('GC', 1241.5)
('HG', 2.7170000000000001)
('HG', 2.6320000000000001)
('HO', 1.581)
('HO', 1.534)
('JY', 0.0089999999999999993)
('JY', 0.0089999999999999993)
('SB', 0.17100000000000001)
('SB', 0.16600000000000001)
('SF', 1.01)
('SF', 1.0009999999999999)
('SP', 2367.0)
('SP', 2353.6999999999998)
('SV', 18.315000000000001)
('SV', 18.059999999999999)
('TY', 124.688)
('TY', 124.23399999999999)
('US', 151.5)
('US', 150.15600000000001)


In [37]:
for market in context['prices'].items:
    print(market.root_symbol, context['prices'][market].high[-context['fifty_five_day_breakout']:].max())
    print(market.root_symbol, context['prices'][market].low[-context['fifty_five_day_breakout']:].min())
#     print(market.root_symbol, context['prices'][market].high[-context['twenty_day_breakout']-1:-1])
#     print(market.root_symbol, context['prices'][market].low[-context['twenty_day_breakout']-1:-1])

('BP', 1.264)
('BP', 1.214)
('CD', 0.76500000000000001)
('CD', 0.73999999999999999)
('CL', 55.146999999999998)
('CL', 47.009999999999998)
('ED', 98.731999999999999)
('ED', 98.635000000000005)
('GC', 1268.337)
('GC', 1197.7449999999999)
('HG', 2.7679999999999998)
('HG', 2.5579999999999998)
('HO', 1.679)
('HO', 1.4830000000000001)
('JY', 0.0089999999999999993)
('JY', 0.0089999999999999993)
('SB', 0.19800000000000001)
('SB', 0.16600000000000001)
('SF', 1.024)
('SF', 0.98899999999999999)
('SP', 2403.0439999999999)
('SP', 2318.0)
('SV', 18.539999999999999)
('SV', 16.824999999999999)
('TY', 125.09399999999999)
('TY', 122.64100000000001)
('US', 152.09399999999999)
('US', 145.81200000000001)


In [ ]:
twenty_day_high = prices.high[-TWENTY_DAY_BREAKOUT:].max()

twenty_day_high

In [ ]:
fifty_five_day_high = prices.high[-FIFTY_FIVE_DAY_BREAKOUT:].max()

fifty_five_day_high

In [ ]:
twenty_day_low = prices.low[-TWENTY_DAY_BREAKOUT:].min()

twenty_day_low

In [ ]:
fifty_five_day_low = prices.low[-FIFTY_FIVE_DAY_BREAKOUT:].min()

fifty_five_day_low

In [ ]:
latest_price = prices.price[-1]

latest_price

In [ ]:
def get_system_one_entry_signal():
    """
    Get 20 day breakout entry signal.
    
    Does not factor in outcome of previous entry signal.
    """
    if latest_price > twenty_day_high:
        print('System one: Long %s %i@%.2f' % (symbols, trade_size, latest_price))
    elif latest_price < twenty_day_low:
        print('System one: Short %s %i@%.2f' % (symbols, trade_size, latest_price))
    else:
        print('System one: No signal')
        
get_system_one_entry_signal()

In [ ]:
def get_system_two_entry_signal():
    """
    Get 55 day breakout entry signal.
    """
    if latest_price > fifty_five_day_high:
        print('System two: Long %s %i@%.2f' % (symbols, trade_size, latest_price))
    elif latest_price < fifty_five_day_low:
        print('System two: Short %s %i@%.2f' % (symbols, trade_size, latest_price))
    else:
        print('System two: No signal')
        
get_system_two_entry_signal()

In [ ]:
latest_price = 240

get_system_one_entry_signal()

In [ ]:
latest_price = 232

get_system_one_entry_signal()

In [ ]:
latest_price = 240

get_system_two_entry_signal()

In [ ]:
latest_price = 231

get_system_two_entry_signal()

In [ ]:
def add_unit():
    """
    Add to position if price exceeds threshold.
    """
    previous_fill_price = 240

    long_threshold = previous_fill_price + volatility / 2
    short_threshold = previous_fill_price - volatility / 2

    if latest_price > long_threshold:
        print('Long %s %i@%.2f' % (symbols, trade_size, latest_price))
    elif latest_price < short_threshold:
        print('Short %s %i@%.2f' % (symbols, trade_size, latest_price))
    else:
        print('No signal')

In [ ]:
latest_price = 241

add_unit()

In [ ]:
latest_price = 239

add_unit()

In [ ]:
latest_price = 239.5

add_unit()

In [ ]:
STOP_MULTIPLIER = 2

entry_price = 240
stop = entry_price - volatility * STOP_MULTIPLIER

stop

In [ ]:
# Re-enter if price hits entry price after stopped out
STOP_MULTIPLIER = 0.5

entry_price = 240
stop = entry_price - volatility * STOP_MULTIPLIER

stop

In [ ]:
TEN_DAY_EXIT_BREAKOUT = 10
TWENTY_DAY_EXIT_BREAKOUT = 20

In [ ]:
ten_day_high = prices.high[-TEN_DAY_EXIT_BREAKOUT:].max()

ten_day_high

In [ ]:
ten_day_low = prices.low[-TEN_DAY_EXIT_BREAKOUT:].min()

ten_day_low

In [ ]:
twenty_day_high = prices.high[-TWENTY_DAY_EXIT_BREAKOUT:].max()

twenty_day_high

In [ ]:
twenty_day_low = prices.low[-TWENTY_DAY_EXIT_BREAKOUT:].min()

twenty_day_low

In [ ]:
def get_system_one_exit_signal():
    if position_type == 'long' and latest_price < ten_day_low \
            or position_type == 'short' and latest_price > ten_day_high:
        print('System one: Close %s' % symbols)
    else:
        print('System one: No signal')

In [ ]:
def get_system_two_exit_signal():
    if position_type == 'long' and latest_price < twenty_day_low \
            or position_type == 'short' and latest_price > twenty_day_high:
        print('System two: Close %s' % symbols)
    else:
        print('System two: No signal')

In [ ]:
position_type = 'long'
latest_price = 236

get_system_one_exit_signal()

In [ ]:
position_type = 'short'
latest_price = 240

get_system_one_exit_signal()

In [ ]:
position_type = 'long'
latest_price = 237

get_system_one_exit_signal()

In [ ]:
position_type = 'long'
latest_price = 232

get_system_two_exit_signal()

In [ ]:
position_type = 'short'
latest_price = 240

get_system_two_exit_signal()

In [ ]:
position_type = 'long'
latest_price = 233

get_system_two_exit_signal()